In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import argparse
import mmcv
import os
import torch
from mmcv import Config, DictAction
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmcv.runner import get_dist_info, init_dist, load_checkpoint

from mmdet3d.apis import single_gpu_test
from mmdet3d.datasets import build_dataloader, build_dataset
from mmdet3d.models import build_detector
from mmdet.apis import multi_gpu_test, set_random_seed
from mmdet.core import wrap_fp16_model
from tools.fuse_conv_bn import fuse_module
from torchsummary import summary

from mmdet3d.apis import init_detector, inference_detector, show_result_meshlab


In [3]:
cfg_file = "../mmdetection3d/configs/3dssd/3dssd_kitti-3d-car.py"
cp_file = "../mmdetection3d/checkpoints/ssd/latest.pth"

In [4]:
try:
    del model
except:
    pass
model = init_detector(cfg_file, cp_file, device='cuda:0')

/home/yzy/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/mmcv/cnn/bricks/conv_module.py:100: UserWarning: ConvModule has norm and bias at the same time
  warnings.warn('ConvModule has norm and bias at the same time')


In [5]:
pcd = '../mmdetection3d/demo/kitti_000008.bin'
result, data = inference_detector(model, pcd)

torch.Size([1, 2, 4096, 32])
torch.Size([1, 2, 4096, 32])
torch.Size([1, 2, 4096, 64])
torch.Size([1, 128, 1024, 32])
torch.Size([1, 128, 1024, 32])
torch.Size([1, 128, 1024, 64])
torch.Size([1, 256, 512, 32])
torch.Size([1, 256, 512, 32])
torch.Size([1, 256, 512, 32])
there


/home/yzy/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/yzy/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/mmcv/utils/misc.py:304: UserWarning: "iou_thr" is deprecated in `nms`, please use "iou_threshold" instead
  f'"{src_arg_name}" is deprecated in '


In [6]:
model

SSD3DNet(
  (backbone): PointNet2SAMSG(
    (SA_modules): ModuleList(
      (0): PointSAModuleMSG(
        (groupers): ModuleList(
          (0): QueryAndGroup()
          (1): QueryAndGroup()
          (2): QueryAndGroup()
        )
        (mlps): ModuleList(
          (0): Sequential(
            (layer0): ConvModule(
              (conv): Conv2d(4, 16, kernel_size=(1, 1), stride=(1, 1))
              (bn): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
              (activate): ReLU(inplace=True)
            )
            (layer1): ConvModule(
              (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
              (bn): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
              (activate): ReLU(inplace=True)
            )
            (layer2): ConvModule(
              (conv): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
              (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, tr

In [6]:
import os
import sys
import copy
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


def knn(x, k):
    inner = -2*torch.matmul(x.transpose(2, 1), x)
    xx = torch.sum(x**2, dim=1, keepdim=True)
    pairwise_distance = -xx - inner - xx.transpose(2, 1)

    idx = pairwise_distance.topk(k=k, dim=-1)[1]   # (batch_size, num_points, k)
    return idx


def get_graph_feature(x, k=20, idx=None):
    batch_size = x.size(0)
    num_points = x.size(2)
    x = x.view(batch_size, -1, num_points)
    if idx is None:
        idx = knn(x, k=k)   # (batch_size, num_points, k)
    device = torch.device('cuda')

    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1)*num_points

    idx = idx + idx_base

    idx = idx.view(-1)

    _, num_dims, _ = x.size()

    x = x.transpose(2, 1).contiguous()   # (batch_size, num_points, num_dims)  -> (batch_size*num_points, num_dims) #   batch_size * num_points * k + range(0, batch_size*num_points)
    feature = x.view(batch_size*num_points, -1)[idx, :]
    feature = feature.view(batch_size, num_points, k, num_dims)
    x = x.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1)

    feature = torch.cat((feature-x, x), dim=3).permute(0, 3, 1, 2).contiguous()

    return feature

In [12]:
# test edge conv

input = torch.randn((1, 64, 4096), device='cuda')

In [13]:
result_1 = get_graph_feature(input)

In [15]:
result_1.shape

torch.Size([1, 128, 4096, 20])

In [27]:
def get_graph_feature_2(x, k=20, idx=None):
    batch_size = x.size(0)
    num_points = x.size(2)
    x = x.view(batch_size, -1, num_points)
    if idx is None:
        idx = knn(x, k=k)   # (batch_size, num_points, k)
    device = torch.device('cuda')

    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1)*num_points

    idx = idx + idx_base

    idx = idx.view(-1)

    _, num_dims, _ = x.size()

    x = x.transpose(2, 1).contiguous()   # (batch_size, num_points, num_dims)  -> (batch_size*num_points, num_dims) #   batch_size * num_points * k + range(0, batch_size*num_points)
    feature = x.view(batch_size*num_points, -1)[idx, :]
    feature = feature.view(batch_size, num_points, k, num_dims).permute(0, 3, 1, 2)
    x = x.transpose(2, 1).contiguous()
    x = x.view(batch_size, num_dims, num_points, 1).repeat(1, 1, 1, k)
    feature = torch.cat((feature-x, x), dim=1)

    return feature

In [28]:
result_2 = get_graph_feature_2(input)


In [29]:
result_2.shape


torch.Size([1, 128, 4096, 20])

In [30]:
result_1 == result_2


tensor([[[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ...